In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/city_day.csv')

In [ ]:
df.isnull().sum()

,0
City,0
Date,0
PM2.5,4598
PM10,11140
NO,3582
NO2,3585
NOx,4185
NH3,10328
CO,2059
SO2,3854


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df_date=pd.DataFrame(df.groupby('Date')['AQI'].mean())
df_date

,AQI
Date,
2015-01-01,217.386318
2015-01-02,214.386318
2015-01-03,162.552985
2015-01-04,188.254498
2015-01-05,189.111641
...,...
2020-06-27,76.210138
2020-06-28,80.902445
2020-06-29,79.863984


In [ ]:
import math

In [ ]:
dataset  = df_date.values
training_data_len = math.ceil(len(dataset)*.8)
training_data_len

1608

In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
scaled_data = sc.fit_transform(dataset)
scaled_data

array([[0.46647406],
       [0.45688034],
       [0.29112208],
       ...,
       [0.02669024],
       [0.00824076],
       [0.02625527]])

In [ ]:
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
num = 60
for i in range(num, len(train_data)):
    x_train.append(train_data[i-num:i , 0])
    y_train.append(train_data[i , 0])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_train.shape

(1548, 60, 1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import datetime

import math
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
plt.style.use("fivethirtyeight")
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer = "adam", loss = "mean_squared_error")

In [ ]:
model.fit(x_train,y_train, batch_size=1, epochs=1)

1548/1548 ━━━━━━━━━━━━━━━━━━━━ 46s 27ms/step - loss: 0.0107


In [ ]:
test_data = scaled_data[training_data_len-60: , :]
x_test = []
y_test = dataset[training_data_len:,:]
for i in range(num, len(test_data)):
      x_test.append(test_data[i-num:i, 0])

In [ ]:
import numpy as np


In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = sc.inverse_transform(predictions)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

3.2021755779660936

In [ ]:
train = df_date[:training_data_len]
valid = df_date[training_data_len:]
valid["Predictions"] = predictions
valid

,AQI,Predictions
Date,,
2019-05-28,135.000000,145.913986
2019-05-29,133.315789,143.200485
2019-05-30,151.129662,140.039597
2019-05-31,147.210526,144.803085
2019-06-01,151.733009,147.453415
...,...,...
2020-06-27,76.210138,82.070335
2020-06-28,80.902445,82.346565
2020-06-29,79.863984,83.709877
